In [3]:
import pandas as pd
import numpy as np
import collections
from sklearn.feature_extraction import text


import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import string

In [4]:
data = pd.read_csv('Evaluation-dataset.csv', header=None)
data.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Tires where delivered to the garage of my choi...,garage service positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Easy Tyre Selection Process, Competitive Prici...",garage service positive,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Very easy to use and good value for money.,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Really easy and convenient to arrange,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,It was so easy to select tyre sizes and arrang...,location positive,value for money positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
10127,"I ordered the wrong tyres, however [REDACTED] ...",refund positive,delivery punctuality positive,refund timescale positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10128,"Good experience, first time I have used [REDAC...",length of fitting positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10129,"I ordered the tyre I needed on line, booked a ...",location positive,delivery punctuality positive,length of fitting positive,value for money positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10130,Excellent service from point of order to fitti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10131,"Seamless, well managed at both ends. I would r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [42]:
print("Shape of the DataFrame:",data.shape)
data.sample(5)

Shape of the DataFrame: (10132, 15)


,Sentence,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,Col_11,Col_12,Col_13,Col_14
527,Good price. Can choose a convenient time and d...,value for money positive,garage service positive,length of fitting positive,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999,Booked in and confirmed A.M appointment via [R...,wait time negative,ease of booking negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6927,Lovely staff efficient purchasing will use the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9583,Tyres arrived and were fitted as expected. No ...,wait time positive,delivery punctuality positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2699,[REDACTED] tyres Quick and easy. Ordered tyres...,ease of booking positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [8]:
sentiment_array = ['Col_'+ str(i) for i in range(1,15)]

new_col = ['Sentence'] + sentiment_array

data.columns = new_col

data.columns

Index(['Sentence', 'Col_1', 'Col_2', 'Col_3', 'Col_4', 'Col_5', 'Col_6',
       'Col_7', 'Col_8', 'Col_9', 'Col_10', 'Col_11', 'Col_12', 'Col_13',
       'Col_14'],
      dtype='object')

# Creating a Dictionary to store the value counts of each subtheme sentence


In [9]:
sent = {}

for i in data.columns:
    if i!='Sentence':
        z = data[i].dropna()
        for l in z:
            if l not in sent.keys():
                sent[l] = 1
            else:
                sent[l]+=1

In [12]:
dict1


{'value for money positive': 4780,
 'garage service positive': 2031,
 'ease of booking positive': 1187,
 'location positive': 1063,
 'length of fitting positive': 657,
 'delivery punctuality positive': 453,
 'tyre quality positive': 434,
 'garage service negative': 423,
 'change of date negative': 277,
 'wait time positive': 274,
 'delivery punctuality negative': 250,
 'advisoragent service positive': 233,
 'ease of booking negative': 227,
 'mobile fitter positive': 225,
 'advisor/agent service positive': 202,
 'value for money negative': 136,
 'wait time negative': 135,
 'damage negative': 127,
 'advisoragent service negative': 125,
 'booking confusion negative': 119,
 'discounts positive': 115,
 'length of fitting negative': 109,
 'extra charges positive': 85,
 'response time negative': 77,
 'late notice negative': 76,
 'incorrect tyres sent negative': 70,
 'advisor/agent service negative': 47,
 'extra charges negative': 46,
 'change of time negative': 42,
 'no stock negative': 42,
 

# Storing the subtheme sentences whose value counts > 30 in a list


In [11]:
import operator
dict1 = dict( sorted(sent.items(), key=operator.itemgetter(1),reverse=True))

list_1 = []

for key, value in dict1.items():
        
    if value>30:
            
        list_1.append(key)

list_1

['value for money positive',
 'garage service positive',
 'ease of booking positive',
 'location positive',
 'length of fitting positive',
 'delivery punctuality positive',
 'tyre quality positive',
 'garage service negative',
 'change of date negative',
 'wait time positive',
 'delivery punctuality negative',
 'advisoragent service positive',
 'ease of booking negative',
 'mobile fitter positive',
 'advisor/agent service positive',
 'value for money negative',
 'wait time negative',
 'damage negative',
 'advisoragent service negative',
 'booking confusion negative',
 'discounts positive',
 'length of fitting negative',
 'extra charges positive',
 'response time negative',
 'late notice negative',
 'incorrect tyres sent negative',
 'advisor/agent service negative',
 'extra charges negative',
 'change of time negative',
 'no stock negative',
 'tyre quality negative',
 'response time positive',
 'facilities positive']

In [13]:
len(list_1)


33


# Creating a function inside which a dataframe is created with the columns of subtheme sentences and their values are mapped from their original position to their position in the created dataframe


In [15]:
def dataModification(data):
    
    data1 = data.iloc[:, 1:]
    
    data2 = pd.DataFrame(index= range(0, data1.shape[0]), columns=list_1)
    
    
    for i, row in data1.iterrows():
        
        l_subthemes = row.dropna().tolist()
        
        for t in l_subthemes:
            
            data2.loc[i][t] = 1
            

    data2 = data2.fillna(0)
    final_data = pd.DataFrame()
    final_data['Sentence'] = data['Sentence']
    for column in data2.columns:
        final_data[column] = data2[column]
    return final_data

In [16]:
final_data = dataModification(data)
final_data.head(20)

,Sentence,value for money positive,garage service positive,ease of booking positive,location positive,length of fitting positive,delivery punctuality positive,tyre quality positive,garage service negative,change of date negative,...,response time negative,late notice negative,incorrect tyres sent negative,advisor/agent service negative,extra charges negative,change of time negative,no stock negative,tyre quality negative,response time positive,facilities positive
0,Tires where delivered to the garage of my choi...,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Easy Tyre Selection Process, Competitive Prici...",1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Very easy to use and good value for money.,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Really easy and convenient to arrange,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,It was so easy to select tyre sizes and arrang...,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,service was excellent. Only slight downside wa...,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,User friendly Website. Competitive Prices. Goo...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Excellent prices and service,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,It was very straightforward and the garage was...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Use of local garage.,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from scikitplot.metrics import plot_confusion_matrix

In [29]:
import string
import re
from collections import Counter
X = final_data['Sentence'].tolist()
X[:2]

['Tires where delivered to the garage of my choice,the garage notified me when they had been delivered. A day and time was arranged with the garage and I went and had them fitted,a Hassel free experience.',
 'Easy Tyre Selection Process, Competitive Pricing and Excellent Fitting Service']

In [30]:
def full_remove(x, removal_list):
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

## Remove digits ##
digits = [str(x) for x in range(10)]
remove_digits = [full_remove(x, digits) for x in X]

## Remove punctuation ##
remove_punc = [full_remove(x, list(string.punctuation)) for x in remove_digits]

## Make everything lower-case and remove any white space ##
sents_lower = [x.lower() for x in remove_punc]
sents_lower = [x.strip() for x in sents_lower]

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

## Remove stop words ##
from nltk.corpus import stopwords
stops = stopwords.words("English")

def removeStopWords(stopWords, txt):
    newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
    return newtxt

sents_processed   = [removeStopWords(stops,x) for x in sents_lower]
sents_processed_2 = [remove_emoji(x) for x in sents_processed]



cnt = Counter()

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
     return " ".join([word for word in str(text).split() if word not in RAREWORDS])
    
sents_processed_3 = [remove_rarewords(x) for x in sents_processed_2]

sents_processed_3[:2]
['tires delivered garage choice garage notified delivered day time arranged garage went fitted hassel free experience',
 'easy tyre selection process competitive pricing excellent fitting service']
sent_processed = pd.DataFrame(sents_processed_3, columns=['Sent_Processed'])
d2 = final_data.iloc[:,1:]
final_data = pd.concat([sent_processed, d2], axis=1)
final_data.head()

,Sent_Processed,value for money positive,garage service positive,ease of booking positive,location positive,length of fitting positive,delivery punctuality positive,tyre quality positive,garage service negative,change of date negative,...,response time negative,late notice negative,incorrect tyres sent negative,advisor/agent service negative,extra charges negative,change of time negative,no stock negative,tyre quality negative,response time positive,facilities positive
0,tires delivered garage choice garage notified ...,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,easy tyre selection process competitive pricin...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,easy use good value money,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,really easy convenient arrange,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,easy select tyre sizes arrange local fitting p...,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Now Vectorizing the words in each sentence using CountVectorizer Method


In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
vect = CountVectorizer(max_df=1.0,stop_words='english') 

In [32]:
X_train,X_test,y_train,y_test = train_test_split(final_data['Sent_Processed'],final_data[final_data.columns[1:]],test_size=0.2)
X_train_vec = vect.fit_transform(X_train)
X_test_vec = vect.transform(X_test)

In [33]:
print(X_train_vec.shape)
print(y_train.shape)
print(X_test_vec.shape)
print(y_test.shape)

(8105, 5574)
(8105, 33)
(2027, 5574)
(2027, 33)


# Model Preparation
NaiveBayesAlgorithm 'OneVsRestClassifier'

In [34]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_vec, y_train)

In [35]:
y_pred_class = nb_classif.predict(X_test_vec)

In [36]:
y_train.head()


,value for money positive,garage service positive,ease of booking positive,location positive,length of fitting positive,delivery punctuality positive,tyre quality positive,garage service negative,change of date negative,wait time positive,...,response time negative,late notice negative,incorrect tyres sent negative,advisor/agent service negative,extra charges negative,change of time negative,no stock negative,tyre quality negative,response time positive,facilities positive
6173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5118,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Evaluating Performance of NaiveBayesAlgorithm 'OneVsRest' Model

In [37]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score,recall_score,precision_score,f1_score

In [38]:
ac = accuracy_score(y_test,y_pred_class)
f1 = f1_score(y_test,y_pred_class,average='micro')
print('Accuracy Score : ', ac)
print('F1-Score : ', f1)

Accuracy Score :  0.3552047360631475
F1-Score :  0.5288461538461539


In [40]:
from sklearn import metrics
print(metrics.classification_report(y_test,y_pred_class))

              precision    recall  f1-score   support

           0       0.71      0.92      0.80       948
           1       0.71      0.25      0.37       419
           2       0.64      0.34      0.45       229
           3       0.86      0.28      0.42       196
           4       0.38      0.04      0.07       123
           5       0.38      0.03      0.06        86
           6       0.50      0.06      0.11       100
           7       0.52      0.34      0.41        93
           8       0.45      0.58      0.51        57
           9       0.00      0.00      0.00        57
          10       0.41      0.45      0.43        56
          11       0.00      0.00      0.00        44
          12       0.46      0.31      0.37        54
          13       0.00      0.00      0.00        46
          14       0.17      0.03      0.05        37
          15       0.50      0.04      0.07        27
          16       0.27      0.10      0.14        31
          17       0.92    

C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\anaconda3\lib\site-packages\s